In [1]:
#Ishank Jain
#20757588
#libraries
from __future__ import print_function
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D       #CNN layers
from keras.datasets import mnist                     #Dataset
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from scipy.ndimage.filters import gaussian_filter
from PIL import Image
from PIL import ImageFilter
import matplotlib.pyplot as plt
from keras import regularizers     #to add l2 regularisation
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
%matplotlib notebook

In [0]:
def resize(images):
    resized_images = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((32, 32))
        img = np.reshape(img, (32, 32, 1))
        img = np.array(img)
        resized_images.append(img)
    return np.array(resized_images)

In [0]:
def resize2(images):
    resized_images = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((32, 32))
        #img = np.reshape(img, (32, 32, 1))
        img = np.array(img)
        resized_images.append(img)
    return np.array(resized_images)

In [0]:
def blurred(images, rad):
    blur = []
    for img in images:
        img = Image.fromarray(img)
        # Use PIL or other library of the sort to rotate
        img = gaussian_filter(img, sigma=rad)        #img = np.array(img)
        img = np.reshape(img, (32, 32, 1))
        blur.append(img)
    return np.array(blur)

In [0]:
def rotates(images, deg):
    rotated = []
    for img in images:
        img = Image.fromarray(img)
        # Use PIL or other library of the sort to rotate
        img = img.rotate(deg)
        #img = np.array(img)
        img = np.reshape(img, (32, 32, 1))
        rotated.append(img)
    return np.array(rotated)

In [0]:
#data Augmentation
def augment_data(x_train, degree):
  image_generator = ImageDataGenerator()
  transofrmation = {'theta': degree}
  new_images = []
  for image in x_train:
    new_images.append(image_generator.apply_transform(image, transformation))
  return new_images


In [0]:
batch_size = 40
num_classes = 10
epochs = 3

model = Sequential()

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
(xtrain, y_train), (xtest, y_test) = mnist.load_data()

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


#aug_data = augment_data(x_train,25)
x_train = resize(x_train)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
xtest = xtest.astype('float32')
xtrain = xtest.astype('float32')

In [24]:
x_test = resize(x_test)
xtest = resize2(xtest)
xtrain = resize2(xtrain)

#normalize data
x_train /= 255
x_test /= 255
xtest /= 255
xtrain /= 255

#data augmentation: adding rotated and blurred data 
xaug1 = rotates(xtrain[1000:8000],30)
xaug2 = blurred(xtest[5000:8000],2)
print(xaug1.shape, xaug2.shape)
x_train = np.concatenate((x_train,xaug1),axis=0)

x_train = np.concatenate((x_train,xaug2),axis=0)
y_train = np.concatenate((y_train,y_train[1000:8000], y_test[5000:8000]), axis = 0)
print(x_train.shape, y_train.shape)

(7000, 32, 32, 1) (3000, 32, 32, 1)
(67000, 32, 32, 1) (67000, 10)


In [25]:

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#block 1
model.add(Conv2D(64,(3, 3),
                 activation='relu', 
                 padding='same',
                 ))
model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 2
model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
   #block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
    #block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))   # 
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))   #
model.add(MaxPooling2D(pool_size=(2, 2)))
    #FC layer and flattening data from 3D to 2D ; REGULARISTION CONSTANT IS 0.01
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(4096,kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
model.add(Dense(4096,kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
model.add(Dense(4096,kernel_regularizer=regularizers.l2(0.0001),activation='relu'))
    #Softmax layer
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=1e-5),
              metrics=['accuracy'])
model.summary


history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[1])
print('Test accuracy:', score[0])



x_train shape: (67000, 32, 32, 1)
67000 train samples
10000 test samples
Train on 67000 samples, validate on 10000 samples
Epoch 1/3
67000/67000 [==============================] - 196s 3ms/step - loss: 1.7236 - acc: 0.6941 - val_loss: 0.8887 - val_acc: 0.9436
Epoch 2/3
67000/67000 [==============================] - 192s 3ms/step - loss: 1.1080 - acc: 0.8630 - val_loss: 0.7419 - val_acc: 0.9684
Epoch 3/3
67000/67000 [==============================] - 191s 3ms/step - loss: 0.9809 - acc: 0.8780 - val_loss: 0.6646 - val_acc: 0.9760
Test loss: 0.976
Test accuracy: 0.66456829662323


In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [30]:
#list of degree to rotate image
degrees=list(range(-45,45,10))
m = list()
for deg in degrees: 
    x_test1 = rotates(xtest, deg)
    score = model.evaluate(x_test1, y_test, verbose=0)
    print("For rotation {} degree, Test loss:{} and Test accuracy:{}".format(deg,score[0], score[1]))
    m.append(score[1])
plt.plot(degrees,m,"ro")
plt.show()

For rotation -45 degree, Test loss:2.7325397869110106 and Test accuracy:0.387
For rotation -35 degree, Test loss:1.934254015159607 and Test accuracy:0.6042
For rotation -25 degree, Test loss:1.179007239341736 and Test accuracy:0.8171
For rotation -15 degree, Test loss:0.7874750715255737 and Test accuracy:0.9341
For rotation -5 degree, Test loss:0.6806519286155701 and Test accuracy:0.9708
For rotation 5 degree, Test loss:0.7114094139099121 and Test accuracy:0.968
For rotation 15 degree, Test loss:1.0218683651924134 and Test accuracy:0.9307
For rotation 25 degree, Test loss:1.8177211179733277 and Test accuracy:0.7477
For rotation 35 degree, Test loss:2.484384663391113 and Test accuracy:0.4411


In [28]:
#list of radius to blur image
radius=list(range(0,7,1))
n = list()
for rad in radius: 
    x_test2 = blurred(xtest, rad)
    score = model.evaluate(x_test2, y_test, verbose=0)
    print("For blur radius {} , Test loss:{} and Test accuracy:{}".format(rad,score[0], score[1]))
    n.append(score[1])
plt.plot(degree,n,"ro")
plt.show()

For blur radius 0 , Test loss:0.66456829662323 and Test accuracy:0.976
For blur radius 1 , Test loss:0.7627343854904175 and Test accuracy:0.9663
For blur radius 2 , Test loss:1.4507092079162598 and Test accuracy:0.875
For blur radius 3 , Test loss:2.3588811222076416 and Test accuracy:0.6686
For blur radius 4 , Test loss:2.726721682739258 and Test accuracy:0.309
For blur radius 5 , Test loss:2.8277939056396484 and Test accuracy:0.219
For blur radius 6 , Test loss:2.8616842834472656 and Test accuracy:0.2167
